In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install timm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import cv2
import torch
import numpy as np
import timm
from scipy.interpolate import RectBivariateSpline
from collections import defaultdict

c:\Users\anura\OneDrive\Desktop\BTP_Code\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#Downloading the model from TorchHub.
midas = torch.hub.load('intel-isl/MiDaS','MiDaS_small')
midas.to('cpu')
midas.eval()

Using cache found in C:\Users\anura/.cache\torch\hub\intel-isl_MiDaS_master


Loading weights:  None


Using cache found in C:\Users\anura/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master


MidasNet_small(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2dSameExport(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
      (3): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU6(inplace=True)
          (se): Identity()
          (conv_pw): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (4): Sequential(
        (0): InvertedResidual(
          (conv_pw): Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(144,

In [8]:
# Load MiDaS transforms
transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = transforms.small_transform

Using cache found in C:\Users\anura/.cache\torch\hub\intel-isl_MiDaS_master


In [7]:
alpha = 0.2
previous_depth = 0.0
depth_scale = 1.0

# Define depth to distance
def depth_to_distance(depth_value, depth_scale):
    return 1.0 / (depth_value * depth_scale)

# Applying exponential moving average filter
def apply_ema_filter(current_depth):
    global previous_depth
    filtered_depth = alpha * current_depth + (1 - alpha) * previous_depth
    previous_depth = filtered_depth  # Update the previous depth value
    return filtered_depth

# Initialize YOLO object detection
yolo = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []

with open("coco.names", "r") as file:
    classes = [line.strip() for line in file.readlines()]

layer_names = yolo.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo.getUnconnectedOutLayers()]

# Create a dictionary to store object trackers with unique colors
object_trackers = {}
next_object_id = 1

# Create a dictionary to store the color assigned to each person
person_colors = defaultdict(lambda: (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255)))

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    # Detect objects using YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    yolo.setInput(blob)
    outputs = yolo.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Initialize a list to store object IDs to be removed
    objects_to_remove = []

    for object_id, tracker_info in object_trackers.items():
        success = tracker_info["tracker"].update(frame)
        if success:
            tracker_box = tracker_info["tracker"].get_position()
            tx, ty, tw, th = [int(tracker_box[i]) for i in range(4)]
            color = tracker_info["color"]
            label = tracker_info["label"]
        else:
            # Handle the case where tracking fails (e.g., object disappeared)
            # You can choose to remove the tracker or take other actions here
            objects_to_remove.append(object_id)
            continue

        object_found = False
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                if x >= tx and y >= ty and x + w <= tx + tw and y + h <= ty + th:
                    object_found = True
                    break

        if not object_found:
            objects_to_remove.append(object_id)
        else:
            # Update the tracker with the new position
            object_trackers[object_id]["tracker"] = tracker_info["tracker"]

            object_frame = frame[y:y + h, x:x + w]  # Extract the object frame

            # Check if the object frame is not empty (width and height are not zero)
            if object_frame.shape[0] > 0 and object_frame.shape[1] > 0:
                # Apply MiDaS depth estimation to the object frame
                object_imgbatch = transform(object_frame).to('cpu')

                with torch.no_grad():
                    prediction = midas(object_imgbatch)
                    prediction = torch.nn.functional.interpolate(
                        prediction.unsqueeze(1),
                        size=object_frame.shape[:2],
                        mode='bicubic',
                        align_corners=False
                    ).squeeze()

                object_output = prediction.cpu().numpy()
                object_output_norm = cv2.normalize(object_output, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

                # Calculate depth for the object
                h_obj, w_obj = object_output_norm.shape
                x_grid_obj = np.arange(w_obj)
                y_grid_obj = np.arange(h_obj)

                spline_obj = RectBivariateSpline(y_grid_obj, x_grid_obj, object_output_norm)
                depth_obj_filt = spline_obj(h_obj / 2, w_obj / 2)
                depth_obj_midas = depth_to_distance(depth_obj_filt, depth_scale)
                depth_obj_filt = (apply_ema_filter(depth_obj_midas) / 10)[0][0]

                # Display the object and its depth with depth at the top of the bounding box
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                text = f"{label} - Depth: {depth_obj_filt:.2f} m"
                cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Remove objects that are no longer being tracked
    for object_id in objects_to_remove:
        del object_trackers[object_id]

    # Detect and track new objects
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])

            object_found = False
            for object_id, tracker_info in object_trackers.items():
                (success, tracker_box) = tracker_info["tracker"].update(frame)
                if success:
                    tx, ty, tw, th = [int(tracker_box[i]) for i in range(4)]
                    if x >= tx and y >= ty and x + w <= tx + tw and y + h <= ty + th:
                        object_found = True
                        break

            if not object_found:
                # Assign a unique color based on the label
                color = person_colors[label]

                # Create a tracker for the new object
                tracker = cv2.TrackerKCF_create()
                tracker.init(frame, (x, y, w, h))

                # Store the tracker along with its color and label
                object_trackers[next_object_id] = {"tracker": tracker, "color": color, "label": label}
                next_object_id += 1

    cv2.imshow('image', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()


AttributeError: 'cv2.TrackerKCF' object has no attribute 'get_position'

: 

Issue Of Global Previous Depth

In [10]:
alpha = 0.2
previous_depth = 0.0
depth_scale = 1.0

# Define depth to distance
def depth_to_distance(depth_value, depth_scale):
    return 1.0 / (depth_value * depth_scale)

# Applying exponential moving average filter
def apply_ema_filter(current_depth):
    global previous_depth
    filtered_depth = alpha * current_depth + (1 - alpha) * previous_depth
    previous_depth = filtered_depth  # Update the previous depth value
    return filtered_depth

# Initialize YOLO object detection
yolo = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []

with open("coco.names", "r") as file:
    classes = [line.strip() for line in file.readlines()]

layer_names = yolo.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo.getUnconnectedOutLayers()]

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    # Detect objects using YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    yolo.setInput(blob)
    outputs = yolo.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])

            object_frame = frame[y:y + h, x:x + w]  # Extract the object frame

            # Check if the object frame is not empty (width and height are not zero)
            if object_frame.shape[0] > 0 and object_frame.shape[1] > 0:
                # Apply MiDaS depth estimation to the object frame
                object_imgbatch = transform(object_frame).to('cpu')

                with torch.no_grad():
                    prediction = midas(object_imgbatch)
                    prediction = torch.nn.functional.interpolate(
                        prediction.unsqueeze(1),
                        size=object_frame.shape[:2],
                        mode='bicubic',
                        align_corners=False
                    ).squeeze()

                object_output = prediction.cpu().numpy()
                object_output_norm = cv2.normalize(object_output, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

                # Calculate depth for the object
                h_obj, w_obj = object_output_norm.shape
                x_grid_obj = np.arange(w_obj)
                y_grid_obj = np.arange(h_obj)

                spline_obj = RectBivariateSpline(y_grid_obj, x_grid_obj, object_output_norm)
                depth_obj_filt = spline_obj(h_obj / 2, w_obj / 2)
                depth_obj_midas = depth_to_distance(depth_obj_filt, depth_scale)
                depth_obj_filt = (apply_ema_filter(depth_obj_midas) / 10)[0][0]

                # Display the object and its depth with depth at the top of the bounding box
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.putText(frame, f"Depth: {depth_obj_filt:.2f} m", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('image', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()

In [11]:
import cv2
import numpy as np
import torch
from scipy.interpolate import RectBivariateSpline

# Assuming you have the necessary imports and definitions for midas and transform

alpha = 0.2
depth_scale = 1.0
previous_depths = {}

# Define depth to distance
def depth_to_distance(depth_value, depth_scale):
    return 1.0 / (depth_value * depth_scale)

# Applying exponential moving average filter
def apply_ema_filter(current_depth, object_index):
    global previous_depths
    if object_index not in previous_depths:
        previous_depths[object_index] = 0.0  # Initialize previous depth for this object

    filtered_depth = alpha * current_depth + (1 - alpha) * previous_depths[object_index]
    previous_depths[object_index] = filtered_depth  # Update the previous depth value for this object
    return filtered_depth

# Initialize YOLO object detection
yolo = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []

with open("coco.names", "r") as file:
    classes = [line.strip() for line in file.readlines()]

layer_names = yolo.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo.getUnconnectedOutLayers()]

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    # Detect objects using YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    yolo.setInput(blob)
    outputs = yolo.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])

            object_frame = frame[y:y + h, x:x + w]  # Extract the object frame

            # Check if the object frame is not empty (width and height are not zero)
            if object_frame.shape[0] > 0 and object_frame.shape[1] > 0:
                # Apply MiDaS depth estimation to the object frame
                object_imgbatch = transform(object_frame).to('cpu')

                with torch.no_grad():
                    prediction = midas(object_imgbatch)
                    prediction = torch.nn.functional.interpolate(
                        prediction.unsqueeze(1),
                        size=object_frame.shape[:2],
                        mode='bicubic',
                        align_corners=False
                    ).squeeze()

                object_output = prediction.cpu().numpy()
                object_output_norm = cv2.normalize(object_output, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

                # Calculate depth for the object
                h_obj, w_obj = object_output_norm.shape
                x_grid_obj = np.arange(w_obj)
                y_grid_obj = np.arange(h_obj)

                spline_obj = RectBivariateSpline(y_grid_obj, x_grid_obj, object_output_norm)
                depth_obj_filt = spline_obj(h_obj / 2, w_obj / 2)
                depth_obj_midas = depth_to_distance(depth_obj_filt, depth_scale)
                depth_obj_filt = (apply_ema_filter(depth_obj_midas, i) / 10)[0][0]

                # Display the object and its depth with depth at the top of the bounding box
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.putText(frame, f"Depth: {depth_obj_filt:.2f} m", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('image', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()


In [ ]:
Gaussian Blur

In [12]:
import cv2
import numpy as np
import torch
from scipy.interpolate import RectBivariateSpline

# Assuming you have the necessary imports and definitions for midas and transform

alpha = 0.2
depth_scale = 1.0
previous_depths = {}

# Define depth to distance
def depth_to_distance(depth_value, depth_scale):
    return 1.0 / (depth_value * depth_scale)

# Applying exponential moving average filter
def apply_ema_filter(current_depth, object_index):
    global previous_depths
    if object_index not in previous_depths:
        previous_depths[object_index] = 0.0  # Initialize previous depth for this object

    filtered_depth = alpha * current_depth + (1 - alpha) * previous_depths[object_index]
    previous_depths[object_index] = filtered_depth  # Update the previous depth value for this object
    return filtered_depth

# Initialize YOLO object detection
yolo = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []

with open("coco.names", "r") as file:
    classes = [line.strip() for line in file.readlines()]

layer_names = yolo.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo.getUnconnectedOutLayers()]

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    # Detect objects using YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    yolo.setInput(blob)
    outputs = yolo.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])

            object_frame = frame[y:y + h, x:x + w]  # Extract the object frame

            # Check if the object frame is not empty (width and height are not zero)
            if object_frame.shape[0] > 0 and object_frame.shape[1] > 0:
                # Apply MiDaS depth estimation to the object frame
                object_imgbatch = transform(object_frame).to('cpu')

                with torch.no_grad():
                    prediction = midas(object_imgbatch)
                    prediction = torch.nn.functional.interpolate(
                        prediction.unsqueeze(1),
                        size=object_frame.shape[:2],
                        mode='bicubic',
                        align_corners=False
                    ).squeeze()

                object_output = prediction.cpu().numpy()
                object_output_norm = cv2.normalize(object_output, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

                # Apply Gaussian blur to the depth map before filtering
                object_output_norm_blur = cv2.GaussianBlur(object_output_norm, (5, 5), 0)

                # Calculate depth for the object
                h_obj, w_obj = object_output_norm_blur.shape
                x_grid_obj = np.arange(w_obj)
                y_grid_obj = np.arange(h_obj)

                spline_obj = RectBivariateSpline(y_grid_obj, x_grid_obj, object_output_norm_blur)
                depth_obj_filt = spline_obj(h_obj / 2, w_obj / 2)
                depth_obj_midas = depth_to_distance(depth_obj_filt, depth_scale)
                depth_obj_filt = (apply_ema_filter(depth_obj_midas, i) / 10)[0][0]

                # Display the object and its depth with depth at the top of the bounding box
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.putText(frame, f"Depth: {depth_obj_filt:.2f} m", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('image', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
